In [1]:
!pip install requests

In [2]:
import base64
import requests
import datetime
from urllib.parse import urlencode
import json

In [3]:
#Spotify Service Authentication

client_id = 'be51e3bfaa534eaf87162873477cc968'
client_secret = '08c8fc0c8f76411c80195068aa995849'

In [4]:
#Putting everything from authorization into Object

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url ='https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" 
        }
    def get_token_data(self):
        return  {
            "grant_type":"client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url #get credentials from developer site
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299): #if request true, will authenticate, was valid_request
            raise Exception ("Could not authenticate client.")
            #return False
        data = r.json()
        token_response_data = r.json()
        now = datetime.datetime.now()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']#seconds
        expires = now + datetime.timedelta(seconds=expires_in)#gives date time relative to now
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        auth_done = self.perform_auth()
        if not auth_done:
            raise Exception("Authentication failed")
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/v1/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return{}
        return r.json()
        
    def get_album(self, _id ):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
        
    def search(self,query,search_type='artist'):
        access_token = self.get_access_token()
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.text


In [5]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.search("Distance", search_type="track")

https://api.spotify.com/v1/search?q=Distance&type=track


'{\n  "tracks" : {\n    "href" : "https://api.spotify.com/v1/search?query=Distance&type=track&offset=0&limit=20",\n    "items" : [ {\n      "album" : {\n        "album_type" : "album",\n        "artists" : [ {\n          "external_urls" : {\n            "spotify" : "https://open.spotify.com/artist/2jku7tDXc6XoB6MO2hFuqg"\n          },\n          "href" : "https://api.spotify.com/v1/artists/2jku7tDXc6XoB6MO2hFuqg",\n          "id" : "2jku7tDXc6XoB6MO2hFuqg",\n          "name" : "Tory Lanez",\n          "type" : "artist",\n          "uri" : "spotify:artist:2jku7tDXc6XoB6MO2hFuqg"\n        } ],\n        "available_markets" : [ "AD", "AE", "AG", "AL", "AM", "AO", "AR", "AT", "AU", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BN", "BO", "BR", "BS", "BT", "BW", "BY", "BZ", "CA", "CD", "CG", "CH", "CI", "CL", "CM", "CO", "CR", "CV", "CW", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FJ", "FM", "FR", "GA", "GB", "GD", "GE", "GH", "GM", "GN", "G